In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # visualization
import seaborn as sns #visualization

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/kaggle/input/kaggle-survey-2021/kaggle_survey_2021_responses.csv'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<a class="anchor" id="0.1"></a>

# <p style="background-color:#F23568;font-family:newtimeroman;color:#FFF9ED;font-size:150%;text-align:center;border-radius:10px 10px;">Searching for the sexiest place for the sexiest job of the 21th Century <br> A female quest</p>
[I. Introduction and background](#1) <br>
[II. A 2021 Overview of diversity in 5 major western economies](#2)<br>
[III. Overview of salaries and education level per country and gender](#3)<br>
[IV. Analysing women inclusion per country and main sectors](#4)<br>
[IV. Exploring salary gaps](#5)<br>
* [IV. a) The female point of view](#5.1)<br>
*  [IV. a) The male point of view](#5.2)<br>

<a class="anchor" id="1"></a>
# <p style="background-color:#E4D5E8;font-family:newtimeroman;color:#2E3C75;font-size:150%;text-align:center;border-radius:10px 10px;">I. Introduction and background 

My name is Caroline, thank you for reading my story.<br> 
18 months ago the made the decision to leave the comfort zone of a well paid job in Paris to take a break in my career to do what I had been longing to do for a while : go back to school and learn something every day. In this intense learning journey, I came accross the exiting field of Data Science that exceeded all my expectation for challenge and thirst for knowledge. <br>
Today, as I am about to resume work and start a new chapter in my life and career, I am curious about the best places to work in Data Science in 2021.
Of course, there is not one size fits all defition of "best" and my criteria are totally subjective. That being said as a black women who worked mostly in male dominated industries, I can say that diversity and inclusion are values that I cherish and that I place high in my ranking system.
In this analysis of the Kaggle 2021 Survey, I explore the state of gender equality in the Data workforce in 5 major western economies through the lense of education and mean salaries in the top recruting sectors for Data scientists.

Note : The choice of countries has been based on the number of respondants and comparability of lifestyles and economies. The focus of the study is on participants who are currently working, therefore unemployed and students participants have not been included.

In [ ]:
#read data 2021
survey_2021=pd.read_csv("../input/kaggle-survey-2021/kaggle_survey_2021_responses.csv",',', dtype='object')

#filter on US, France, Germany,UK, Spain and select only those who are working
countries_filter=(survey_2021['Q3']=='France')|(survey_2021['Q3']=='Germany')|(survey_2021['Q3']=='United States of America')|(survey_2021['Q3']=='United Kingdom of Great Britain and Northern Ireland')|(survey_2021['Q3']=='Spain')
workforce_filter=(survey_2021['Q5']!='Student')&(survey_2021['Q5']!='Currently not employed')
workforce_2021=survey_2021.loc[(workforce_filter & countries_filter),:].reset_index(drop=True)

# renaming countries
countries_dic={'France':'France', 'Germany':'Germany', 'Spain': 'Spain', 
               'United Kingdom of Great Britain and Northern Ireland':'UK',
               'United States of America':'US'}
workforce_2021['Country']=workforce_2021['Q3'].map(countries_dic)
#list of countries to make analysis easier
countries=workforce_2021['Country'].sort_values().unique()

# converting salary ranges to numerical categories to enable stats estimation
wage_dic={'$0-999': 0.5,'1,000-1,999':1.5, '10,000-14,999':12.5, '100,000-124,999':112.5,
       '125,000-149,999':137.5, '15,000-19,999':17.5, '150,000-199,999':175, '2,000-2,999':2.5,
       '20,000-24,999':22.5, '200,000-249,999':225 , '25,000-29,999':27.5, '3,000-3,999':3.5,
       '30,000-39,999':35, '300,000-499,999':400, '4,000-4,999':4.5, '40,000-49,999':45,
       '5,000-7,499':6.5, '50,000-59,999':55, '60,000-69,999':65, '7,500-9,999':8.75,
       '70,000-79,999':75, '80,000-89,999':85, '90,000-99,999':95, '>$1,000,000':1000 }
workforce_2021['Wages']=workforce_2021['Q25'].map(wage_dic)

#regrouping categories based on similarity and/or lack of representation in the data 
def gen_ed_sec_map(x):
    if x=='Nonbinary'or x=='Prefer to self-describe': 
        return 'NA'
    elif x=='Professional doctorate'or x=='Doctoral degree':
        return 'Doctors'
    elif x=='No formal education past high school' or x=='Some college/university study without earning a bachelor’s degree':
        return 'No college/university degree'
    elif x=='Accounting/Finance'or x=='Insurance/Risk Assessment':
        return 'Finance/Insurance'
    else:
        return x

workforce_2021['Genders']=workforce_2021['Q2'].map(gen_ed_sec_map)
workforce_2021['Education']=workforce_2021['Q4'].map(gen_ed_sec_map)
workforce_2021['Sector']=workforce_2021['Q20'].map(gen_ed_sec_map)

#style and format
w_color='#F23568'
m_color='#2E3C75'
na_color='#E4D5E8'
bg_color= '#F2F2F2'
plt.style.use('default')

<a class="anchor" id="2"></a>
# <p style="background-color:#E4D5E8;font-family:newtimeroman;color:#2E3C75;font-size:150%;text-align:left;border-radius:10px 10px;">II. Overview of diversity in 5 major western economies in 2021 
#### ***The art of being caught between a rock and a hard place.*** <br>

In this section, my objective was to determine and compare the overall level of inclusion in the data science community in the five selected country.<br>
If I were to chose the best place to work in Data Science based on the sole percentage of women in the workforce,that would be an impossible choice. The United States and the United Kindom are the diversity champions with barely 2 women for 8 men.


In [ ]:
#calculating proportion of each gender per country
gender_filter=workforce_2021['Genders']!='Prefer not to say' #filter on those who answered the gender question
wf_genders=round(pd.crosstab(workforce_2021.loc[gender_filter,'Country'],
                       workforce_2021.loc[gender_filter, 'Genders'],margins=True,normalize=0)*100,2)
wf_genders=wf_genders.sort_values('Man', ascending=False).reset_index()

#plotting
fig,ax = plt.subplots(figsize=(8, 4))
 
ax.barh( wf_genders['Country'], -wf_genders['Man'],color=m_color)
ax.barh( wf_genders['Country'], wf_genders['Woman'],color=w_color)
ax.barh( wf_genders['Country'], wf_genders['NA'], left=wf_genders['Woman'], color=na_color)
ax.legend(labels=['Man','Woman','Not applicable*'],bbox_to_anchor=(0.45, -0.25), ncol=3, frameon=0)

#customizing visualization
for location in ['left','right', 'bottom']:
        ax.spines[location].set_visible(False)
ax.spines['top'].set_color(bg_color) 
ax.set_ylim(-0.5,6)
ax.set_yticks(np.arange(6)) 
ax.set_yticklabels(['Spain', 'France', 'Germany','Countries\nAverage', 'UK', 'US'], color="grey")
ax.tick_params( left=False, size=5,direction='in', pad=5, color='grey') # customizing ticks
ax.set_xticks((-100,-90,-80,0,10,20)) 
ax.set_xticklabels(('',90,80,0,10,20), color='grey')
ax.xaxis.tick_top()

# customizing the global average
ax.barh( 3-0.01, -(wf_genders.loc[3,'Man'])+0.6,color='white',height=0.65)
ax.barh( 3-0.01, wf_genders.loc[3,'Woman']-0.5,color='white', height=0.65)
ax.axvline(x=-0.2, ymin=0.49, ymax=0.59,
               linewidth=2, color=m_color)
#adding titles and comments
ax.text(s='Major Western Economies keep losing the inclusion battle in Data',
        x=-110, y=8.5, weight='bold', fontsize='16')
ax.text(s='In 2021, less than 2 data workers out of 10 are Women.',x=-110,y=8, fontsize=14)
ax.text(s='% of workers per gender',x=-100,y=6.75, fontsize=12, color="grey", fontstyle='italic')

ax.annotate(text='With only 1 female for 9 males \nSpain is the less inclusive\nof the five countries',
            xy=(10,0), xytext=(35, 0.5),fontstyle='italic',backgroundcolor =bg_color,
            fontsize=10.5, color='grey', arrowprops=dict(arrowstyle="->", 
                                shrinkA=25, shrinkB=15,color='grey',alpha=0.35,
                                patchA=None, patchB=None,mutation_scale=10,
                                connectionstyle="angle,angleA=-90,angleB=180,rad=50"))

ax.annotate(text='With 21% of women and 78% of men\n US is the most diverse data workforce',
            xy=(21,5), xytext=(45, 6),fontstyle='italic',backgroundcolor =bg_color,
            fontsize=10.5, color='grey', arrowprops=dict(arrowstyle="->", 
                                shrinkA=15, shrinkB=15,color='grey',alpha=0.35,
                                patchA=None, patchB=None,mutation_scale=10,
                                connectionstyle="angle,angleA=-90,angleB=180,rad=50"))
ax.text(s='*This analysis only include respondants who answered the question on gender\n The "not applicable" category includes "Non binary" and "Prefer to self-describe answers".',
        x=-100,y=-3.25, fontsize=8, color="grey", fontstyle='italic')
plt.show()

# <p style="background-color:#E4D5E8;font-family:newtimeroman;color:#2E3C75;font-size:150%;text-align:left;border-radius:10px 10px;">III. Overview of salaries and education level per country and gender <a class="anchor" id="3"></a>
#### ***When studying harder won't necessarily pay off***

In this section I compare mean salaries between men and women in each of the selected country. 
As a French citizen my initial thought when I first saw the data was 'if you want to get return on investment for your higher education you'd better expatriate. However in terms of gender equality Germany appears to be the promise land for frustration : 95% of women have at least a master degree and the country has the highest percentage of female Doctors, but in this country Men are paid on average 54% more than women in the Data Science community.
US and the UK have much lower barrier entries in terms of education despite high gender gaps in salaries. 
Of course if my only criterion was Salary I would apply for a green card immediately.


In [ ]:
# Estimating mean salaries per country
wage_filter=workforce_2021['Wages']!=1000 # removing this salary range as they are outliers
wf_mean_sal=workforce_2021.loc[wage_filter].pivot_table(index=['Q3'], columns='Genders', values='Wages', aggfunc='mean').sort_values('Woman')
#Estimating Gender gap in mean salaries
wf_mean_sal['Gen_gap']=round((wf_mean_sal['Man']-wf_mean_sal['Woman'])/wf_mean_sal['Woman']*100,1)
#proportion of each gender per country and degree
ed_filter=workforce_2021['Education']!='I prefer not to answer' #filter on those who answered the education level question
wf_ed_gen=pd.crosstab([workforce_2021['Q3'],workforce_2021.loc[(workforce_2021['Genders']!='NA')& (gender_filter),'Genders']], 
                       workforce_2021.loc[ed_filter,'Education'], normalize=0, margins=True)*100
wf_ed_gen=wf_ed_gen.iloc[:,[1,2,0,3]] # reorder columns from highest degree to lowest degree

wf_man_ranges=['45-50','50-55','70-75','70-75', '112.5-125'] # male salary ranges labels for visualization
wf_woman_ranges=['30-35', '35-40', '45-50','50-55', '80-85'] # female salary ranges labels for visualization

# plotting salaries
fig,ax = plt.subplots(figsize=(14, 5))
ax.scatter(x=wf_mean_sal.index, y=wf_mean_sal['Man'], marker="_", color=m_color, s=1200, linewidth=3)
ax.scatter(x=wf_mean_sal.index, y=wf_mean_sal['Woman'], marker="_", color=w_color, s=1200,linewidth=3)
ax.set_ylim(0,130)
ax.set_xticks([])
ax.set_yticks([])

ax.tick_params(top=False, left=False) # removing ticks

for location in [ 'top', 'right','bottom', 'left']: # removing axis
        ax.spines[location].set_visible(False)
        
for i, m, mr,g,w,wr,c in zip(range(0,5), wf_mean_sal['Man'],wf_man_ranges, 
                             wf_mean_sal['Gen_gap'],wf_mean_sal['Woman'],wf_woman_ranges,
                             ['Spain','France','Germany','UK','US'] ):
    ax.annotate(text=mr, xy=(i,m), xytext=(i-0.15, m), fontsize=10, alpha=0.25, va='center', ha='right')
    ax.annotate(text=str(g)+'%', xy=(i,(m+w)/2), xytext=(i+0.25, 10), fontsize=12, alpha=0.5, 
                arrowprops=dict(arrowstyle="-", 
                                shrinkA=15, shrinkB=40,color='grey',
                                patchA=None, patchB=None,mutation_scale=10,
                                connectionstyle="angle,angleA=-90,angleB=180,rad=3"))
    ax.text(s=c,x=i,y=130, fontsize=14, weight='bold', color='grey')
    ax.annotate(text=wr, xy=(i,w), xytext=(i-0.15, w), fontsize=10, alpha=0.25, va='center', ha='right')

# comments
ax.annotate(text='The French paradox:\n the highest overall level of education\n but the lowest salaries after Spain.\n(KUSD 50-55 for men\nKUSD 35-40 for women)',
            xy=(0.65,55), xytext=(0, 90),fontstyle='italic',backgroundcolor=bg_color,
            fontsize=9, color='grey', arrowprops=dict(arrowstyle="->", 
                                shrinkA=0, shrinkB=5,color='grey',alpha=0.35,
                                patchA=None, patchB=None,mutation_scale=10,
                                connectionstyle="angle3,angleA=80,angleB=-40"))
ax.annotate(text='German female workers are the\nmost educated population\nbut in Germany men are\npaid 54% more than women.',
            xy=(2.2,10), xytext=(1.5, -20),fontstyle='italic',backgroundcolor=bg_color,
            fontsize=9, color='grey', arrowprops=dict(arrowstyle="->", 
                                shrinkA=15, shrinkB=0,color='grey',alpha=0.35,
                                patchA=None, patchB=None,mutation_scale=10,
                                connectionstyle="angle,angleA=-90,angleB=180, rad=20"))
ax.text (s="Men get much higher return on investment for higher education",x=-0.5,y=160, weight='bold', fontsize=16)
ax.text(s="In the top 4 economies, women are more qualified than men but men are paid up to 54% more.",
         x=-0.5, y=152, fontsize=12)
ax.text(s='Estimated mean salary per country and gender (KUSD)',  x=-0.5, y=140, fontsize=10, fontstyle='italic', color='grey')
#plotting education
fig2,(ax1,ax2,ax3,ax4,ax5) = plt.subplots(1,5,figsize=(16, 1), sharey=True)

for ax, i in zip([ax2,ax3,ax1,ax4,ax5], range(0,5)):
    for location in ['left','right', 'top', 'bottom']: # removing axis
        ax.spines[location].set_visible(False)
    
    for j,c in zip (range(0,4),[[m_color,w_color],[m_color,w_color],bg_color,bg_color]):
        ax.barh( ['Man', 'Woman'], wf_ed_gen.iloc[i*2:(i+1)*2,j], left=wf_ed_gen.iloc[i*2:(i+1)*2,0:j].sum(axis=1),color=c)
     
       
    ax.tick_params(bottom=False, left=False)
    ax.set_xticks((0,25,50,75,100)) 
    ax.set_xticklabels((0,25,50,75,100), color='grey') 
    ax.set_yticks((1,0)) 
    ax.set_yticklabels(('Woman','Man'), color='grey') 
    
ax1.text(s="% of workers with a Master's or Doctor degree",x=-30,y=2, fontsize=10, fontstyle='italic', color='grey')


plt.show()

# <p style="background-color:#E4D5E8;font-family:newtimeroman;color:#2E3C75;font-size:150%;text-align:left;border-radius:10px 10px;">IV. Analysing women inclusion per country and main sectors <a class="anchor" id="4"></a>
#### ***'When they go low, we go high'... but this is not necessarily a good thing when you talk about salaries and number of female workers*** <br>

In this section I consider the 5 main sectors employing more than 77% of the Survey respondants.
The sector having the highest percentage of women is Education it is also the sector that have the lowest mean salaries.
It is interesting to see that in the top 4 economies the sectors that have the highest percentage of women are also the ones paying the lowest, and these sectors are not necessarily the same in all countries.

In [ ]:
#computing mean salary per main sector and country
main_sectors_filter=((workforce_2021['Sector']=='Computers/Technology')|
                    (workforce_2021['Sector']=='Academics/Education')|
                    (workforce_2021['Sector']=='Medical/Pharmaceutical')|
                    (workforce_2021['Sector']=='Government/Public Service')|
                    (workforce_2021['Sector']=='Manufacturing/Fabrication')|
                    (workforce_2021['Sector']=='Finance/Insurance'))
wf_mean_sal_sec=workforce_2021.loc[wage_filter & main_sectors_filter].pivot_table(index=['Sector', 'Country'],  values='Wages', aggfunc='mean').reset_index()

#computing mean salary per gender per main sector and country
wf_mean_sal_sec_gen=workforce_2021.loc[wage_filter & main_sectors_filter].pivot_table(index=['Sector', 'Country'], columns='Genders', 
                                                                                      values='Wages', aggfunc='mean')[['Man','Woman']].reset_index()
#renaming columns
wf_mean_sal_sec_gen.rename(columns={'Man':'Wages_Man','Woman': 'Wages_Woman'}, inplace=True)
#Calculating gender Gap in absolute value
wf_mean_sal_sec_gen['Wages_gap']=(wf_mean_sal_sec_gen['Wages_Man']-wf_mean_sal_sec_gen['Wages_Woman'])
#calculating proportion of male and female per industry and country
wf_indus_gen=pd.crosstab([workforce_2021['Sector'],workforce_2021['Country']],
                         workforce_2021.loc[(workforce_2021['Genders']!='NA')& (gender_filter),'Genders'], normalize=0)
wf_main_indus_gen=wf_indus_gen.loc[['Computers/Technology','Academics/Education','Finance/Insurance',#selecting main sectors
                                    'Medical/Pharmaceutical','Government/Public Service',
                                    'Manufacturing/Fabrication']].reset_index()
#merging 3 dataframes, reordering and renaming columns
merge_sec_sal_gen=wf_mean_sal_sec.merge(wf_main_indus_gen).merge(wf_mean_sal_sec_gen)
merge_sec_sal_gen['Sector_country']=merge_sec_sal_gen['Sector']+' '+merge_sec_sal_gen['Country']
merge_sec_sal_gen=merge_sec_sal_gen.iloc[:,[0,1,8,2,5,6,7,3,4]]

In [ ]:
indus_palette=({'Computers/Technology':'#0DD92C','Academics/Education':'#F23E2E','Finance/Insurance':'#F2E638',
            'Medical/Pharmaceutical':'#9168F5','Government/Public Service':'#C2DCF2','Manufacturing/Fabrication':'#F205CB'}) # color palette for sectors
#plotting story 3
story3=merge_sec_sal_gen.copy()

gs_kw = dict(width_ratios=[12,4], height_ratios=[3,3,3,3,3],wspace=0.3)
fig,((ax1,ax6),(ax2,ax7),(ax3,ax8),(ax4,ax9),(ax5,ax10)) = plt.subplots(5,2,figsize=(14, 8), 
                                                                         gridspec_kw=gs_kw)
for ax_g1, ax_g2, c in zip ((ax4,ax3,ax5,ax2,ax1),(ax9,ax8,ax10,ax7,ax6),countries):
    for location in ['left','right', 'top']:
        ax_g1.spines[location].set_visible(False)
        ax_g2.spines[location].set_visible(False)
    ax_g1.spines['bottom'].set_color(bg_color) 
    ax_g2.spines['bottom'].set_color(bg_color) 
    
    sns.scatterplot(y='Sector' ,x='Wages', 
                    data=story3.loc[story3['Country']==c].sort_values('Wages', ascending=False), 
                    hue='Sector', ax=ax_g1, legend=False, palette=indus_palette,marker='^', s=80) 
      
    ax_g1.set_xlim(20,140)
    ax_g1.set_ylim(6,-1)
    ax_g1.set_ylabel('')
    ax_g1.set_yticks([])
    ax_g1.set_xticks([])
    ax_g1.set_xlabel('')
    ax_g1.tick_params(bottom=False, left=False)
    ax_g1.text(s=c,
        x=-20, y=3, fontsize='10', color='grey', weight='bold')
    
    ax_g2.barh( (story3.loc[story3['Country']==c].sort_values('Wages')['Sector']), 
               -(story3.loc[story3['Country']==c].sort_values('Wages')['Man']*100),color=m_color)
    ax_g2.barh( (story3.loc[story3['Country']==c].sort_values('Wages')['Sector']),
               (merge_sec_sal_gen.loc[merge_sec_sal_gen['Country']==c].sort_values('Wages')['Woman']*100),color=w_color)
    
    ax_g2.tick_params(bottom=False, left=False)
    ax_g2.set_xlim(-100,30)
    ax_g2.set_xticks([])
    ax_g2.set_yticks((0,1,2,3,4,5))
    ax_g2.set_yticklabels((story3.loc[story3['Country']==c].sort_values('Wages')['Sector']), color='grey', fontsize=8)

# comments and customization
ax5.set_xticks(np.arange(20,141,20))
ax5.set_xticklabels((20, 40,60,80,100,120,140), color='grey', fontsize=8)
ax5.set_xlabel('(KUSD)', color='grey')
ax10.set_xticks((-100,-90,-80,0,10,20))
ax10.set_xticklabels(('',90,80,0,10,20), color='grey', fontsize=8)

ax1.text(s='When inclusion goes high, salaries go low ',  x=-20, y=-5.5, fontsize=16,weight='bold')
ax1.text(s='In the top 4 economies, the sectors that have the highest proportion of female workers pays the lowest',
         x=-20, y=-3.75, fontsize=13)
ax1.text(s='Estimated mean salary per country and sector',  x=-20, y=-1, fontsize=10, fontstyle='italic', color='grey')
ax6.text(s='% of male and female workers',  x=30, y=6, fontsize=10, fontstyle='italic', color='grey', ha='right')

ax5.annotate(text='Spain is the only country \nwhere sectors paying the highest salaries\n have the highest percentage of women',
            xy=(60,3), xytext=(60, 3),fontstyle='italic',
            fontsize=9, color='grey', backgroundcolor=bg_color)

ax2.annotate(text='UK has both the highest proportion\nof women and the lowest mean\nsalary in the medical sector.',
            xy=(45,4), xytext=(0, 2),fontstyle='italic',backgroundcolor=bg_color,
            fontsize=9, color='grey', arrowprops=dict(arrowstyle="->", 
                                shrinkA=25, shrinkB=15,color='grey',alpha=0.35,
                                patchA=None, patchB=None,mutation_scale=10,
                                connectionstyle="angle,angleA=-90,angleB=180,rad=50"))
plt.show()

<a class="anchor" id="5"></a>
# <p style="background-color:#E4D5E8;font-family:newtimeroman;color:#2E3C75;font-size:150%;text-align:left;border-radius:10px 10px;">V. Exploring salary gaps 

### ***From the female stand point*** <a class="anchor" id="5.1"></a>

If like me, you would define a good place to work as one that would combine (1) a salary in higher ranges, (2) a good level of diversity and (3)gender equality, you might be disappointed. In 2021, such a place does not seem to exist. I want to stay optimistic so I am going to say that this place does not exist yet.
To finish on a positive note, despite counting barely 5% of women, UK Finance/insurance sector is actually paying female more than men on average. 


In [ ]:
story4=merge_sec_sal_gen.copy().sort_values('Wages_Woman', ascending=False)

gs_kw = dict(width_ratios=[3,12], height_ratios=[3],wspace=0.7)
fig,(ax1,ax2) = plt.subplots(1,2,figsize=(15, 8), gridspec_kw=gs_kw, sharey=True)
                                                                         
for location in ['left','right', 'top']:
    ax1.spines[location].set_visible(False)
    ax2.spines[location].set_visible(False)
    
#plotting left plot
ax1.barh( (story4['Sector_country']), (story4['Woman'])*100,color=w_color)
ax1.axvline(x=wf_genders.copy().loc[3,'Woman'], ymin=0, ymax=0.96, color='grey', alpha=0.25)
#plotting right   
sns.scatterplot(y='Sector_country' ,x='Wages_Man', 
                    data=story4, 
                    ax=ax2, legend=False, color=m_color)
sns.scatterplot(y='Sector_country' ,x='Wages_Woman', 
                    data=story4, 
                    ax=ax2, legend=False, color=w_color)


#plotting horizontal lines
data_hl=story4.loc[(story4['Sector_country']!='Government/Public Service_France')&
                   (story4['Sector_country']!='Manufacturing/Fabrication_Spain')].reset_index()
for i in range(0,28):
    plt.plot(data_hl.loc[i, ['Wages_Woman', 'Wages_Man']], (i,i), color='grey', alpha=0.25,linewidth=1)

#customization and comments
ax1.yaxis.tick_right()
ax1.set_yticks(np.arange(0,30))
ax1.set_yticklabels((story4['Sector_country']), color='grey', ha='center', position=(1.7,0), fontsize=8)
ax1.tick_params( right=False, size=5, pad=10, color=bg_color) # customizing ticks
ax1.set_xticks(np.arange(0,40,10))  
ax1.set_xticklabels((0,10,20,30), color='grey', fontsize=8)
ax1.spines['bottom'].set_color(bg_color)  
ax2.set_xlabel('(KUSD)', color='grey')
ax2.set_xticks(np.arange(20,141,20))
ax2.set_xticklabels((20, 40,60,80,100,120,140), color='grey', fontsize=8)
ax2.tick_params( left=False, right=False, size=5, pad=5, color=bg_color) # customizing ticks
ax2.spines['bottom'].set_color(bg_color)    

ax1.text(s='Low gap in wages does not necessarily mean more diversity.',  x=-20, y=-6, fontsize=16,weight='bold')
ax1.text(s='Inclusion and gaps in mean salaries per main sector and location',
         x=-20, y=-4.75, fontsize=14)
ax1.text(s='% of women in workforce',  x=0, y=-2, fontsize=10, fontstyle='italic', color='grey')
ax2.text(s='Mean salary per gender',  x=100, y=-2, fontsize=10, fontstyle='italic', color='grey')
ax2.annotate(text='French Tech\n pays men and woman\n equally but is\nless inclusive\nthan Tech in the US and UK',
            xy=(60,11), xytext=(100, 18),fontstyle='italic',backgroundcolor=bg_color,
            fontsize=9, color='grey', arrowprops=dict(arrowstyle="->", 
                                shrinkA=15, shrinkB=20,color='grey',alpha=0.35,
                                patchA=None, patchB=None,mutation_scale=10,
                                connectionstyle="angle,angleA=-90,angleB=0,rad=50"))
ax2.annotate(text='UK Finance sector\npays women more\n but is one of the less inclusive\n place to work',
            xy=(80,3), xytext=(10, 7),fontstyle='italic',backgroundcolor=bg_color,
            fontsize=9, color='grey', arrowprops=dict(arrowstyle="->", 
                                shrinkA=15, shrinkB=20,color='grey',alpha=0.35,
                                patchA=None, patchB=None,mutation_scale=10,
                                connectionstyle="angle,angleA=-90,angleB=0,rad=50"))
ax2.text(s='(*)',  x=80, y=1, fontsize=8, fontstyle='italic', color='grey')
ax2.text(s='(*) There are only 2 female respondants in this sector/location. \n The estimated mean salary might not be representative of the actual salary of the sector',
         x=0, y=35, fontsize=8, fontstyle='italic', color='grey')
plt.show()

### ***From the male stand point*** <a class="anchor" id="5.2"></a>
Needless to say that gender equality won't be achieved without men...guys can we make it together ?

In [ ]:
#data for plot
story5=merge_sec_sal_gen.copy().sort_values('Wages_Man', ascending=False)
data_hl=story5.loc[(story5['Sector_country']!='Government/Public Service_France')&
                   (story5['Sector_country']!='Manufacturing/Fabrication_Spain')].reset_index()

gs_kw = dict(width_ratios=[6,6], height_ratios=[3],wspace=0.7)
fig,(ax1,ax2) = plt.subplots(1,2,figsize=(15, 8), gridspec_kw=gs_kw, sharey=True)
                                                                         
for location in ['left','right', 'top']:
    ax1.spines[location].set_visible(False)
    ax2.spines[location].set_visible(False)
    
#plotting left plot
sns.scatterplot(y='Sector_country' ,x='Wages_Man', data=story5, ax=ax1, legend=False, color=m_color, alpha=0.1)
sns.scatterplot(y='Sector_country' ,x='Wages_Woman', data=story5, ax=ax1, legend=False, color=w_color, alpha=0.1)
sns.scatterplot(y='Sector_country' ,x='Wages_Man', data=story5.loc[[6,4,9,23,14,11]], ax=ax1, legend=False, 
                color=m_color)
sns.scatterplot(y='Sector_country' ,x='Wages_Woman', data=story5.loc[[6,4,9,23,14,11]], ax=ax1, legend=False,
                color=w_color)
    
#plotting right   
sns.scatterplot(y='Sector_country' ,x='Wages_Man', data=story5, ax=ax2, legend=False, color=m_color, alpha=0.1)
sns.scatterplot(y='Sector_country' ,x='Wages_Woman', data=story5, ax=ax2, legend=False, color=w_color, alpha=0.1)
sns.scatterplot(y='Sector_country' ,x='Wages_Man', data=story5.loc[[13,5,17,2,1,28,18]], ax=ax2,
                legend=False, color=m_color)
sns.scatterplot(y='Sector_country' ,x='Wages_Woman', data=story5.loc[[13,5,17,2,1,28,18]], ax=ax2, 
                legend=False, color=w_color)

#plotting horizontal lines
for i in range(0,30):
    ax1.plot(data_hl.loc[i, ['Wages_Woman', 'Wages_Man']], (i,i), color=bg_color,alpha=0.25,linewidth=1)
    ax2.plot(data_hl.loc[i, ['Wages_Woman', 'Wages_Man']], (i,i), color=bg_color,alpha=0.25,linewidth=1)
for i in [10,11,1,14,0,9]:
    ax1.plot(data_hl.loc[i, ['Wages_Woman', 'Wages_Man']], (i,i), color='grey', alpha=0.25,linewidth=1)
for i in [8,18,20,27,26,23,16]:
    ax2.plot(data_hl.loc[i, ['Wages_Woman', 'Wages_Man']], (i,i), color='grey', alpha=0.25,linewidth=1)

#customization and comments
ax1.yaxis.tick_right()
ax2.tick_params( left=False)
ax1.set_ylabel('')
for ax in (ax1,ax2):
    ax.set_xlabel('(KUSD)', color='grey')
    ax.set_xticks(np.arange(20,141,20))
    ax.set_xticklabels((20, 40,60,80,100,120,140), color='grey')
    ax.tick_params( right=False, size=5, pad=5, color=bg_color) # customizing ticks
    ax.spines['bottom'].set_color(bg_color) 

ax1.set_yticks(np.arange(0,30))
ax1.set_yticklabels((story5['Sector_country']), color='#595959', ha='center', position=(1.3,0), fontsize=8)

ax1.text(s='Where men are paid high, gaps are high. Where men are paid low, gaps are low.',
         x=-20, y=-6, fontsize=16,weight='bold')
ax1.text(s='Apart from UK Finance sector, small salary gaps are exclusively achieved where men are paid less.',
         x=-20, y=-4.75, fontsize=14)
ax1.text(s='Salary gaps above KUSD 30',  x=60, y=-2, ha='center',fontsize=10, fontstyle='italic', color='grey')
ax2.text(s='Salary gaps below KUSD 10',  x=60, y=-2, ha='center',fontsize=10, fontstyle='italic', color='grey')

ax1.annotate(text='In germany,Female workers\nin Finance/assurance\nare paid K58.6 less\nthan their male colleagues',
            xy=(30,9), xytext=(-20, 5),fontstyle='italic',backgroundcolor=bg_color,
            fontsize=9, color='grey', arrowprops=dict(arrowstyle="->", 
                                shrinkA=0, shrinkB=20,color='grey',alpha=0.35,
                                patchA=None, patchB=None,mutation_scale=10,
                                connectionstyle="angle,angleA=-90,angleB=-30,rad=50"))
ax2.annotate(text='French Tech pays men\nand woman equally but\noffers subtanstially lower\nwages compared to what \nTech pays to men in the US,\nUK or Germany',
            xy=(65,18), xytext=(80, 24),fontstyle='italic',backgroundcolor=bg_color,
            fontsize=9, color='grey', arrowprops=dict(arrowstyle="->", 
                                shrinkA=0, shrinkB=20,color='grey',alpha=0.35,
                                patchA=None, patchB=None,mutation_scale=10,
                                connectionstyle="angle,angleA=-90,angleB=0,rad=60"))
plt.show()

# <p style="background-color:#E4D5E8;font-family:newtimeroman;color:#2E3C75;font-size:150%;text-align:center;border-radius:10px 10px;">Final note 

Thank you for reading thus far. 
I had a lot of fun exploring this dataset so I hope you had as much as fun reading my story.

**<p style="background-color:#F23568;font-family:newtimeroman;color:#F2F2F2;font-size:150%;text-align:center;border-radius:10px 10px;">Don't hesitate to like, comment and upvote !**

[table of Contents](#0.1)